In [94]:
import pandas as pd

In [95]:
all_post = pd.read_csv("Sentiment_Analysis_Dataset.csv", header=0, delimiter=",", error_bad_lines=False, index_col=0)

Skipping line 8836: expected 4 fields, saw 5

Skipping line 535882: expected 4 fields, saw 7



In [96]:
all_post = all_post.drop("SentimentSource", axis=1)

In [97]:
all_post = all_post.reset_index(drop=True)

In [98]:
all_post = all_post[0:all_post.size / 3]

In [112]:
test = all_post[all_post.size / 3: all_post.size / 3 * 2]

In [62]:
def clean_post(raw_post):
    import re
    from nltk.corpus import stopwords
    from nltk.stem.snowball import SnowballStemmer
    #Download package stopwords and stemmer use:
    #import nltk
    #nltk.download()
    
    #url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    #raw_post = re.sub(url_pattern, '', raw_post, flags=re.MULTILINE)
    #raw_post = HTMLParser.HTMLParser().unescape(raw_post.decode('utf-8')) #&quot - to symbol representation"
    
    letters_only = re.sub("[^a-zA-Z]", " ", raw_post)
    lower_case = letters_only.lower()
    words = lower_case.split()
    #stemmer = SnowballStemmer("english")
    #words = [stemmer.stem(word) for word in words]
    return words

In [93]:

import nltk.data
#nltk.download() 

In [64]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [65]:
def post_to_sentences(post, tokenizer):
    raw_sentences = tokenizer.tokenize(post.decode('utf-8').strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(clean_post(raw_sentence))
    return sentences

In [ ]:
sentences = []

for i in xrange(0, all_post.SentimentText.size):
    post = all_post.SentimentText[i]
    sentences += post_to_sentences(post, tokenizer)

In [ ]:
print len(sentences)

In [72]:
num_features = 300                    
min_word_count = 40                        
num_workers = 6
context = 10
downsampling = 1e-3

In [73]:
from gensim.models import word2vec

In [74]:
model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count = min_word_count, window = context, sample = downsampling)

In [75]:
model.init_sims(replace=True)

In [76]:
model_name = "300features_40minwords_10context"
model.save(model_name)

In [128]:
#model.most_similar("haha")

In [84]:
from sklearn.cluster import KMeans
import time

start = time.time() # Start time

word_vectors = model.syn0
num_clusters = word_vectors.shape[0] / 5

kmeans_clustering = KMeans( n_clusters = num_clusters )
idx = kmeans_clustering.fit_predict( word_vectors )

end = time.time()
elapsed = end - start
print "Time taken for K Means clustering: ", elapsed, "seconds."

Time taken for K Means clustering:  736.212476015 seconds.


In [85]:
word_centroid_map = dict(zip( model.index2word, idx ))

In [87]:
# For the first 10 clusters
for cluster in xrange(0,10):
    #
    # Print the cluster number  
    print "\nCluster %d" % cluster
    #
    # Find all of the words for that cluster number, and print them out
    words = []
    for i in xrange(0,len(word_centroid_map.values())):
        if( word_centroid_map.values()[i] == cluster ):
            words.append(word_centroid_map.keys()[i])
    print words


Cluster 0
[u'robin']

Cluster 1
[u'nova', u'anderson', u'wonderland']

Cluster 2
[u'knitting', u'delivering', u'length', u'layers', u'concrete', u'metal', u'motion', u'rubber', u'lighting']

Cluster 3
[u'elsewhere', u'monthly', u'location', u'meantime', u'reserve', u'import', u'publish', u'ban', u'sample', u'mob', u'mailbox', u'exchange', u'payment', u'badge', u'section']

Cluster 4
[u'rb', u'su', u'ao', u'tw', u'td', u'whee', u'og', u'vc', u'dl', u'nos', u'gk', u'za', u'sy', u'ir', u'ik', u'ju', u'jd', u'twi', u'twt', u'ku', u'kt', u'mw', u'tg', u'lu', u'lookbook', u'imeem', u'oi', u'pi']

Cluster 5
[u'tomarrow', u'tomoz']

Cluster 6
[u'cramps', u'chills', u'ache', u'pains', u'sneeze', u'aches', u'body', u'bug', u'aching', u'snot', u'kills']

Cluster 7
[u'bourbon', u'leftovers']

Cluster 8
[u'loudly', u'stupidly']

Cluster 9
[u'ohhhhh']


In [89]:
def create_bag_of_centroids( wordlist, word_centroid_map ):
    num_centroids = max( word_centroid_map.values() ) + 1
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    return bag_of_centroids

In [102]:
clean_train_posts = []
for post in all_post.SentimentText:
    clean_train_posts.append(clean_post(post))

In [114]:
clean_test_posts = []
for post in test.SentimentText:
    clean_test_posts.append(clean_post(post))

In [115]:
import numpy as np
train_centroids = np.zeros( (all_post.SentimentText.size, num_clusters), dtype="float32" )

# Transform the training set reviews into bags of centroids
counter = 0
for review in clean_train_posts:
    train_centroids[counter] = create_bag_of_centroids( review, word_centroid_map )
    counter += 1

In [117]:
# Repeat for test reviews 
test_centroids = np.zeros(( test.Sentiment.size, num_clusters), dtype="float32" )

counter = 0
for post in clean_test_posts:
    test_centroids[counter] = create_bag_of_centroids(post, word_centroid_map )
    counter += 1

In [ ]:
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)

forest = forest.fit(train_centroids, all_post.Sentiment)
result = forest.predict(test_centroids)

# Write the test results 
output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
output.to_csv( "BagOfCentroids.csv", index=False, quoting=3 )